## YOLO v8 - Detección de máscarillas

In [1]:
# IMPORTANDO LIBRERIAS NECESARIAS Y PROBANDO LA INSTALACION DE YOLO CORRECTAMENTE
import os # LIBRERIA PARA MANEJAR ARCHIVOS Y DIRECTORIOS
import json # LIBRERIA PARA MANEJAR ARCHIVOS JSON
import glob # LIBRERIA PARA MANEJAR ARCHIVOS
import torch # LIBRERIA DE PYTORCH
import shutil # LIBRERIA PARA MOVER ARCHIVOS
import subprocess # LIBRERIA PARA EJECUTAR COMANDOS DE CONSOLA
import ultralytics # LIBRERIA DE YOLO
from PIL import Image # LIBRERIA PARA MANEJAR IMAGENES
import xml.etree.cElementTree as ET # LIBRERIA PARA MANEJAR ARCHIVOS XML

# DEFINIENDO TU DIRECTORIO DE TRABAJO
root_dir = os.getcwd()


In [2]:
# VISUALIZANDO EL CONTENIDO DE LOS XML DE LAS ANOTACIONES DE LAS IMAGENES (ETIQUETADO)
with open(os.path.join("./dataset/annotations","maksssksksss9.xml")) as f:
    contents = f.read()
    print(contents)


<annotation>
    <folder>images</folder>
    <filename>maksssksksss9.png</filename>
    <size>
        <width>267</width>
        <height>400</height>
        <depth>3</depth>
    </size>
    <segmented>0</segmented>
    <object>
        <name>mask_weared_incorrect</name>
        <pose>Unspecified</pose>
        <truncated>0</truncated>
        <occluded>0</occluded>
        <difficult>0</difficult>
        <bndbox>
            <xmin>148</xmin>
            <ymin>75</ymin>
            <xmax>201</xmax>
            <ymax>133</ymax>
        </bndbox>
    </object>
    <object>
        <name>without_mask</name>
        <pose>Unspecified</pose>
        <truncated>0</truncated>
        <occluded>0</occluded>
        <difficult>0</difficult>
        <bndbox>
            <xmin>27</xmin>
            <ymin>78</ymin>
            <xmax>56</xmax>
            <ymax>106</ymax>
        </bndbox>
    </object>
</annotation>


### 1- PREPARACIÓN DEL DATASET

In [3]:
# DEFINIENDO LOS DIRECTORIOS DE IMAGENES, ANOTACIONES Y ETIQUETAS
# DEFINIENDO LAS CLASES -> ETIQUETAS DE NUESTRO MODELO
classes = ["without_mask", "with_mask", "mask_weared_incorrect"]
image_dir = os.path.join("./dataset","images")
input_dir = os.path.join("./dataset","annotations")
output_dir = os.path.join("./dataset","labels")
# VERIFICANDO SI EL DIRECTORIO DE SALIDA EXISTE, SI NO, LO CREA
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)
# ---------------------------------------------------------------------------------------------------------------------

# CONVIERTE EL ARCHIVO XML DE CADA IMG A TXT DEVOLVIENDO ASI X, Y, ANCHO Y ALTO DE LA IMAGEN DE SU MARCO DELIMITADOR
def xml_to_yolo_bbox(bbox, w, h):
    # bbox (bounding box) -> MARCO O CAJA DELIMITADORA EN FORMATO DE COORDENADAS (XMIN, YMIN, XMAX, YMAX)
    # bbox ES UN RECTANGULO QUE RODEA UN OBJETO EN UNA IMAGEN PARA IDENTIFICAR SU POSICION Y TAMAÑO
    centro_x = ((bbox[2] + bbox[0]) / 2) / w
    centro_y = ((bbox[3] + bbox[1]) / 2) / h
    # ESTO ES PARA NORMALIZAR LOS VALORES DE ANCHO Y ALTO DE LA IMAGEN EN 1 y 0
    ancho = (bbox[2] - bbox[0]) / w 
    alto = (bbox[3] - bbox[1]) / h

    return [centro_x, centro_y, ancho, alto]

# ---------------------------------------------------------------------------------------------------------------------

def generating_labels(file):
    result = []
    # ANALIZANDO EL CONTENIDO DEL ARCHIVO XML
    tree = ET.parse(file)
    # OBTENIENDO LAS PROPIEDADES DE LA IMAGEN
    root = tree.getroot()
    # EXTRAYENDO ANCHO Y ALTO DE LA IMAGEN DESDE SUS PROPEDADES
    width = int(root.find("size").find("width").text)
    height = int(root.find("size").find("height").text)
    # OBTENIENDO UNA LISTA CON EL OBJETO DE CADA IMAGEN
    for obj in root.findall("object"):
        # EXTRAYENDO EL NOMBRE DE LA CLASE (ETIQUETA) DE CADA OBJETO DE IMAGEN
        label = obj.find("name").text

        # VERIFICA SI HAY NUEVAS CLASES Y LAS AGREGA A LA LISTA DE CLASES
        if label not in classes:
          classes.append(label)
        index = classes.index(label) # 0:SIN MASCARILLA 1:CON MASCARILLA 2:MASCARILLA MAL PUESTA

        pil_bbox = [int(x.text) for x in obj.find("bndbox")] # OBTENIENDO EL MARCO DE LA IMAGEN
        yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height) # AJUSTANDO EL MARCO DE LA IMAGEN A UN MARCO PARA YOLO

        # CONVIRTIENDO LOS DATOS DEL OBJETO DEL MARCO DE LA IMAGEN DE YOLO A STRING
        bbox_string = " ".join([str(x) for x in yolo_bbox]) #
        result.append(f"{index} {bbox_string}")
    if result:
        # CONVIRTIENDO EL ARCHIVO XML A UN ARCHIVO TXT DE YOLO
        with open(os.path.join(output_dir, f"{filename}.txt"), "w", encoding = "utf-8") as f:
            f.write("\n".join(result))
            
# ---------------------------------------------------------------------------------------------------------------------   

# OBTENIENDO TODOS LOS FILES DE TIPO XML DEL INPUT DIRECTORY
files = glob.glob(os.path.join(input_dir, "*.xml"))
# RECORRIENDO CADA ARCHIVO XML Y BUSCANDO SU IMAGEN CORRESPONDIENTE
for file in files:
    basename = os.path.basename(file)
    filename = os.path.splitext(basename)[0]
    if not os.path.exists(os.path.join(image_dir, f"{filename}.png")):
        print(f"{filename} la imagen NO existe!")
        continue
    generating_labels(file)
# GENERANDO LAS CATEGORIAS O CLASSES DE NUESTRO MODELO
with open("./classes.txt", "w", encoding = "utf-8") as f:
    f.write(json.dumps(classes))
    
# ---------------------------------------------------------------------------------------------------------------------

# MOSTRANDO LAS ETIQUETAS DE CLASSES GENERADAS 
with open('./classes.txt') as f:
    contents = f.read()
    print(contents)

["without_mask", "with_mask", "mask_weared_incorrect"]


In [4]:
# MOSTRANDO LAS ETIQUETAS DE LAS IMAGENES EN FORMATO YOLO
with open(os.path.join("./dataset/labels","maksssksksss2.txt")) as f:
    contents = f.read()
    print(contents)

1 0.21625 0.19137931034482758 0.0925 0.09310344827586207
1 0.415 0.20862068965517241 0.06 0.09310344827586207
1 0.625 0.17758620689655172 0.06 0.1206896551724138
2 0.87375 0.16551724137931034 0.0825 0.11724137931034483


### CREANDO LAS CARPETAS NECESARIAS

In [ ]:
# CREANDO LAS CARPETAS DE ENTRENAMIENTO, VALIDACION Y PRUEBA
if os.path.isfile(os.path.join("./dataset/data")):
    print("ESTA CARPETA YA EXISTE!")
else:
    os.mkdir(os.path.join("./dataset/data"))
    os.mkdir(os.path.join("./dataset/data/train"))
    os.mkdir(os.path.join("./dataset/data/val"))
    os.mkdir(os.path.join("./dataset/data/test"))
    os.mkdir(os.path.join("./dataset/data/train/images"))
    os.mkdir(os.path.join("./dataset/data/train/labels"))
    os.mkdir(os.path.join("./dataset/data/test/images"))
    os.mkdir(os.path.join("./dataset/data/test/labels"))
    os.mkdir(os.path.join("./dataset/data/val/images"))
    os.mkdir(os.path.join("./dataset/data/val/labels"))
    print("CARPETAS CREADAS CON EXITO!")

In [6]:
# GUARDANDO LOS NOMBRES DE LAS IMAGENES EN UNA LISTA
img_names = []

for i in os.listdir(os.path.join("./dataset/images")):
    img_name = i[:-4] #ELIMINANDO LA EXTENSION DE LAS IMAGENES
    img_names.append(img_name)

In [7]:
# SEPARANDO EL CONJUNTO DE DATOS EN ENTRENAMIENTO, PRUEBA Y VALIDACION
def segmentation_data(main_txt_file, main_img_file, train_size, test_size, val_size):
    # ASIGNANDO LAS IMAGENES A CADA CARPETA SEGUN EL ENTRENAMIENTO QUE SE DESEE
    list_of_trainings = [[0, train_size, "train"], # RANGO DE TRAINING
                         [train_size , train_size + test_size, "test"], # RANGO DE TESTING
                         [train_size + test_size , train_size + test_size + val_size, "val"]] # RANGO DE VALIDATION
    # DATOS DE ENTRENAMIENTO
    for i in list_of_trainings:
        for j in range(i[0], i[1]):
            destination_txt = os.path.join(f'./dataset/data/{i[2]}/labels', img_names[j]+".txt")
            destination_png = os.path.join(f'./dataset/data/{i[2]}/images', img_names[j]+".png")
            shutil.copy(f'{main_txt_file}/{img_names[j]}.txt', destination_txt)
            shutil.copy(f'{main_img_file}/{img_names[j]}.png', destination_png)

# GENERANDO LOS DATOS DE ENTRENAMIENTO, PRUEBA Y VALIDACION        
segmentation_data(os.path.join("./dataset/labels"),os.path.join("./dataset/images"), 350, 150, 150) # train_size - test_size - val_size

In [8]:
# EL ARCHIVO YAML CONTIENE LA CONFIGURACION DEL MODELO DE YOLO COMO, PARAMETROS , RUTAS DE LOS DATOS, ETC
# SE UTILIZA PARA CONFIGURAR EL MODELO DE YOLO
yaml_text = """train: ./train/images
val: ./val/images
nc: 3
names: ["without_mask", "with_mask", "mask_weared_incorrect"]"""

try:
    with open(os.path.join("./dataset/data/data.yaml"),"w") as model_config:
        model_config.write(yaml_text)
        print("CREANDO EL ARCHIVO YAML")
except:
    os.mkdir(os.path.join("./dataset/data/data.yaml"))
    with open(os.path.join("./dataset/data/data.yaml"),"w") as model_config:
        model_config.write(yaml_text)
        print("SE CREO LA CARPETA Y EL ARCHIVO YAML")

CREANDO EL ARCHIVO YAML


# INFORMACIÓN
* IoU（Intersection over Union）En el campo de la visión por computadora, IoU se utiliza típicamente para medir la precisión de las predicciones del modelo. La forma de calcular IoU es dividiendo el área de intersección entre el área de unión de la región objetivo predicha (rectángulo) y la región objetivo real. Es decir, IoU = intersección / unión.

* En la detección de objetos, cuanto mayor sea el valor de IoU entre el cuadro del objetivo predicho por el modelo y el cuadro del objetivo real, más precisa será la predicción del modelo. Por lo general, cuando IoU supera un umbral predefinido (por ejemplo, 0.5 o 0.7), se puede considerar que la predicción es exitosa en la detección.
 
## mAP50(B) -> Precisión para objetos grandes:
* Significa "Mean Average Precision at IoU 0.50 for Large Objects".
Es el promedio de la precisión (AP) calculada para objetos grandes con un umbral de IoU de 0.50.
mAP es el promedio de AP para diferentes clases en un modelo, mientras que mAP50(B) se centra en objetos más grandes.

## mAP50-95(B) -> Precisión para objetos grandes:
* Significa "Mean Average Precision across IoU 0.50 to 0.95 for Large Object".
Es el promedio de la precisión (AP) calculada para objetos grandes en un rango de IoU de 0.50 a 0.95.

## Precision(B) -> Precisión de el reconocimiento:
* Es la precisión calculada para objetos grandes.
Representa la proporción de muestras correctamente predichas como positivas (Verdaderos Positivos) sobre todas las muestras predichas como positivas (Verdaderos Positivos + Falsos Positivos).

## Recall(B) -> Muestras correctamente identificadas:
* Es el recall calculado para objetos grandes.
Representa la proporción de muestras correctamente identificadas como positivas (Verdaderos Positivos) sobre todas las muestras positivas (Verdaderos Positivos + Falsos Negativos).
Indica cuántos de los objetivos reales se detectaron correctamente.

## Dfl_loss (Detection Focal Loss) -> Detección de objetos difíciles:
* Es una función de pérdida utilizada en tareas de detección de objetos para abordar el problema de desequilibrio de clases.
Se enfoca en aumentar la pérdida para ejemplos positivos difíciles, lo que ayuda al modelo a concentrarse en la detección de objetos difíciles.

## Box_loss (Box Regression Loss) -> Detección sobre el marco delimitador de la imagen:
* Es una función de pérdida utilizada para evaluar y optimizar las predicciones del modelo sobre las coordenadas de las cajas delimitadoras (bounding boxes).
Mide la diferencia entre las coordenadas predichas de la caja delimitadora y las coordenadas reales, y busca minimizar esta diferencia para mejorar la precisión de las predicciones de las cajas delimitadoras.

## Cls_loss (Class Classification Loss) -> Precisión de la detección sobre los marcos delimitadores de la imagen:
* Es una función de pérdida utilizada en modelos YOLO para evaluar la precisión de clasificación de cada caja delimitadora.
Calcula la pérdida comparando las etiquetas de clase predichas con las etiquetas de clase reales, con el objetivo de mejorar el rendimiento de clasificación del modelo.

### VISUALIZAR EJECUCIÓN EN TIEMPO REAL

In [28]:
# EJECUTANDO EL COMANDO DE ENTRENAMIENTO Y MOSTRANDO LOS RESULTADOS EN TIEMPO REAL
def view_real_time_output(command):
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    print("SALIDA DEL COMANDO:")
    for byte_line in iter(process.stdout.readline, b''):
        decoded_line = byte_line.decode('utf-8', errors='replace').strip()
        print(decoded_line)
        
    # ESPERANDO A QUE EL PROCESO DE ENTRENAMIENTO TERMINE
    process.wait()

### ENTRENAMIENTO

In [ ]:

# ESTABLECIENDO EL DIRECTORIO DE RESULTADOS DEL ENTRENAMIENTO
if not os.path.isdir(os.path.join(root_dir,"./dataset/result")):
    os.mkdir(os.path.join(root_dir,"./dataset/result"))
    print("CREANDO DIRECTORIO DE RESULTADOS")
os.chdir(os.path.join(root_dir,"./dataset/result"))
data_path= os.path.join(os.path.dirname(os.getcwd()), "data", "data.yaml")

# DEFINIENDO LOS HIPERPARAMETROS PARA EL ENTRENAMIENTO
epochs = 50 # EPOCAS DE ENTRENAMIENTO
img_size = 640 # TAMANO DE LA IMAGEN A ENTRENAR
batch_size = 20 # TAMAÑO DEL LOTE DE IMAGENES

# ENTRENANDO EL MODELO DE YOLO
command = f"yolo train model=yolov8n.pt data={data_path} epochs={epochs} imgsz={img_size} batch={batch_size} device=0"
view_real_time_output(command)

### DETECCIONES

In [ ]:
# RECONFIGURANDO EL DIRECTORIO A LA RAIZ PARA EVITAR CONFLICTOS
os.chdir(root_dir)
os.chdir(os.path.join(root_dir,"./dataset/result/runs/detect/train"))
train_id = '' # ID DEL ENTRENAMIENTO, VACIO EN  CASO DE SER EL PRIMERO
train_path= os.path.join(os.path.dirname(os.getcwd()), 'train'+str(train_id)+'/weights', 'best.pt')
os.chdir(root_dir)
img_test_path = os.path.join(root_dir,"./dataset/data/test/images")

# EJECUTANDO EL COMANDO DE PREDICCION Y MOSTRANDO LOS RESULTADOS EN TIEMPO REAL
command = f'yolo predict model={train_path} source={img_test_path}'
view_real_time_output(command)

### TEST CON DATOS REALES

In [ ]:
# PROBANDO CON IMAGENES REALES
os.chdir(root_dir)
img_test_path = os.path.join(root_dir,"./dataset/test.png")
command = f'yolo predict model={train_path} source={img_test_path}'
view_real_time_output(command)


In [ ]:
# VERIFICANDO LA PREDICCION DE LA IMAGEN DE PRUEBA
os.chdir(root_dir)
Image.open(os.path.join(root_dir,"./runs/detect/predict/test.png"))